#### going to run the code line by line

"#eval" refers to evaluating tf code and translating the output to a numpy file through the function tf_eval

"#checkpoint" refers to converting a np array evaulated through tf_eval into a tf tensor and use it in the code. This way the code will not run from the very beginning

Fist let's create y_map for two images, one with boats and on without

In [1]:
# imports
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import image as mpimg
import random
#import pyplot as pl
import time

/Users/margheritarosnati/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/margheritarosnati/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [57]:
def tf_eval(tf_tensor,timed=0):
    if timed:
        startTime = time.time()
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        np_output = tf_tensor.eval(session=sess)
    if timed:
        print("time:",time.time()-startTime)
    return np_output

In [3]:
# functions to create training files
def read_images(filenames):
    '''
    Function applied to every data entry to load the data. The output of this is the input format
    to the model.
    :param filename: filename
    :param y_map: y_data (passes straight through)
    :return:
    '''

    imgs = []
    for i in range(len(filenames)):
        print(filenames[i])
        imgs.append(mpimg.imread(filenames[i]))
    imgs = np.asarray(imgs)
    return imgs


def get_y_data(filename):
    '''
    Converts a filename into an array, with one channel for each boat in the image
    :param filename: filename string
    :return: np array of size [h, w, n_boats_in_this_image]
    '''

    ground_truths = y_raw[y_raw.ImageId == filename]
    array_of_coords = np.array(ground_truths[['lt_x', 'lt_y', 'rb_x', 'rb_y']])
    # array_of_coords is of shape [n_boxes, 4]

    n_boxes = array_of_coords.shape[0]
    y_map = np.zeros((768, 768, n_boxes))
    y_reg = np.zeros((768, 768, n_boxes, 4))

    for box_idx in range(n_boxes):
        # Loop over amount of boats per image ~ of order 10.
        box = array_of_coords[box_idx, :]
        y_map[box[0]:box[2], box[1]:box[3], box_idx] = 1

        # Add boat box coordinates to y_reg
        y_reg[:, :, box_idx, 0] = round((box[2] + box[0]) / 2)  # x-centre
        y_reg[:, :, box_idx, 1] = round((box[3] + box[1]) / 2)  # y-centre
        y_reg[:, :, box_idx, 2] = box[2] - box[0]  # width
        y_reg[:, :, box_idx, 3] = box[3] - box[1]  # height

    return y_map, y_reg

def padder(list_of_arr):
    '''
    Pads each list of boat maps so all have the same depth (which is the max amount of
    boats across all images) and creates a numpy array of the result.
    :param list_of_arr: list of arrays, each shaped [768, 768, n_boats_in_this_image]
    :return: numpy array of shape [len(list_of_arr), h, w, maximum_n_boats]
    '''

    maximum_n_boats = max([x.shape[2] for x in list_of_arr])
    dim_arr = list_of_arr[0].shape

    n_box_max = maximum_n_boats

    b = np.zeros([len(list_of_arr), dim_arr[0], dim_arr[1], maximum_n_boats])
    for i, arr in enumerate(list_of_arr):
        b[i,:, :, :arr.shape[2]] = arr
    return b

def padder_coord(list_of_arr):
    '''
    Pads each list of boat maps so all have the same depth (which is the max amount of
    boats across all images) and creates a numpy array of the result.
    :param list_of_arr: list of arrays, each shaped [768, 768, n_boats_in_this_image, depth] (depth typically 4)
    :return: numpy array of shape [len(list_of_arr), h, w, maximum_n_boats, depth]
    '''

    maximum_n_boats = max([x.shape[2] for x in list_of_arr])
    dim_arr = list_of_arr[0].shape
    depth = dim_arr[-1]

    b = np.zeros([len(list_of_arr), dim_arr[0], dim_arr[1], maximum_n_boats, depth])
    for i, arr in enumerate(list_of_arr):
        b[i, :, :, :arr.shape[2], :] = arr
    return b

In [ ]:
labels_file =  "../data/labels/boxes_v2.csv"
y_raw = pd.read_csv(labels_file)

sub_input = ['0a148697c.jpg' , '0a0aeea56.jpg']
training_data_path = "../data/train_sample/"
filenames = [training_data_path + x for x in sub_input]

In [ ]:
# dont run me twice
input_img = read_images(filenames)

y_data = []
y_reg = []
for file in sub_input:
    out_y_data = get_y_data(file)
    y_data.append(out_y_data[0])
    y_reg.append(out_y_data[1])

y_data = padder(y_data)
y_reg = padder_coord(y_reg)

In [ ]:
# dont run me twice
# let's save these files so that we wont have to do this again
y_data_path = "debug/y_data.pkl"
y_reg_path = "debug/y_reg.pkl"
np.save(y_data_path, y_data)
np.save(y_reg_path, y_reg)

In [5]:
# run me from time 2
y_data_path = "debug/y_data.pkl"
y_reg_path = "debug/y_reg.pkl"
y_data = np.load(y_data_path+".npy")
y_reg = np.load(y_reg_path+".npy")

sub_input = ['0a148697c.jpg' , '0a0aeea56.jpg']
training_data_path = "../data/train_sample/"
filenames = [training_data_path + x for x in sub_input]
input_img = read_images(filenames)

../data/train_sample/0a148697c.jpg
../data/train_sample/0a0aeea56.jpg


In [51]:
input_img.shape

(2, 768, 768, 3)

# Now let's run the code "line by line":

In [52]:
x = tf.convert_to_tensor(input_img, tf.float32)
y_map = tf.convert_to_tensor(y_data, tf.float32)
y_reg = tf.convert_to_tensor(y_reg, tf.float32)

In [53]:
x.shape

TensorShape([Dimension(2), Dimension(768), Dimension(768), Dimension(3)])

## Data

### expand_anchor_shapes_for_reg

In [63]:
# First let's try 2 anchors
anchor_shapes = [(21,21)]

In [64]:
#anchor_shapes = [(21,21), (21, 41), (41,21), (41, 81), (81, 41), (51,51), (151,81), (81, 151), (101,101), (201,201)]
n_anchors = len(anchor_shapes)
y_anchors = np.zeros((1, 768, 768, n_anchors, 4))
x = np.arange(768)
X, Y = np.meshgrid(x, x)
y_anchors[:, :, :, :, 0] = np.reshape(Y, (1, 768, 768, 1))
y_anchors[:, :, :, :, 1] = np.reshape(X, (1, 768, 768, 1))
i = 0
for anchor_shape in anchor_shapes:
    y_anchors[:, :, :, i, 2] = np.ones((1, 768, 768)) * anchor_shape[0]  # width
    y_anchors[:, :, :, i, 3] = np.ones((1, 768, 768)) * anchor_shape[1]  # length
    i += 1
# this last part needs to stay here with this code construction: we change the first dimension of
# the reg_anchors to the batch size implicitly (unsure if it works if done explicitly)
reg_anchors = tf.tile(y_anchors, multiples=[tf.shape(y_reg)[0], 1, 1, 1, 1])
reg_anchors = tf.cast(reg_anchors, tf.float32)

In [ ]:
# evaluate
np_reg_anchors = tf_eval(reg_anchors)

In [ ]:
# np_reg_anchors generates a matrix that has for every image and every anchor, 
# the centre, height and width of the anchor for each pxel
print(np_reg_anchors.shape)
print(np_reg_anchors[0,0,0,0,:]) # anchor shape is (21,21) centered at (0,0)
print(np_reg_anchors[0,5,10,0,:]) # anchor shape is (21,21) centered at (5,10)

### expand_y

In [9]:
# Here is where we unwrap the y masks to be IOU maps and then maps that match the loss.
y_class = []
selected_boat_index = []
iou_mask = []
iou_average_for_summary = []
pos_mask = []
neg_mask = []
n_box = tf.shape(y_map)[-1]

note: the code below iterates per anchor shape. We'll first run it on the first shape

In [ ]:
i=0
anchor shape = (21,21)

In [ ]:
############ NOTE: dont run me twice unless absolutely necessary!!
# Loop over number of anchors ~ of order 9
# The y_map is of shape [batch_size, h, w, max_n_boats]
# The kenerl we convolve with is of shape [anchor_shape[0], anchor_shape[1], max_n_boats, max_n_boats]
# where every entry is 0 except for [:,:, i, i] for all i.
# (I think) this is equivalent to running a seperate "all ones" [anchor_shape[0], anchor_shape[1], 1]
# kernel over each of the max_n_boats inputs.
# TODO: double (triple, quadruple...) check above logic.

anchor = tf.zeros((anchor_shape[0], anchor_shape[1], n_box, n_box))
diagonal = tf.ones((anchor_shape[0], anchor_shape[1], n_box))
anchor_area = anchor_shape[0]*anchor_shape[1]

# Assigns ones to anchor[:,:,i,i] (https://www.tensorflow.org/api_docs/python/tf/linalg/set_diag)
anchor = tf.linalg.set_diag(anchor, diagonal)

# Calculates the intersection of anchor with each gt map in y_map simulatenously (as above)
intersection = tf.nn.conv2d(y_map, anchor, strides=[1, 1, 1, 1], padding='SAME')

# union is the area of the map (per map layer, and per batch entry) + the anchor area (in 2d)
# TODO: check that minusing intersection does so entry wise.
union = tf.reduce_sum(y_map, [1, 2], keepdims=True) + anchor_area - intersection
old_ious = tf.divide(intersection, union)

In [ ]:
############ NOTE: dont run me twice unless absolutely necessary!!
# evaluate
np_ious = tf_eval(ious)

In [ ]:
############ NOTE: dont run me twice unless absolutely necessary!!
# IOUs take some time to evaluate so we are going to save the instance and run TF from here
ious_path = "debug/ious.pkl"
np.save(ious_path, np_ious)

In [10]:
############ NOTE: take me instead of running twice the guy above
ious_path = "debug/ious.pkl"
np_ious = np.load(ious_path+".npy")
ious = tf.convert_to_tensor(np_ious, tf.float32)

In [ ]:
print(np_ious.shape) # (2, 768, 768, 6) where 6 is the number of boats
print(np.sum(np_ious[1,:])) # this should be zero and it is
for i in range(6):
    print("sum:",np.sum(np_ious[0,:,:,i]), "\tmax", np.amax(np_ious[0,:,:,i]))

In [13]:
# checkpoint
ious = tf.convert_to_tensor(np_ious, tf.float32)

In [11]:
iou_positive_threshold = 0.5
iou_negative_threshold = 0.1

In [14]:
max_iou_over_ground_truth = tf.reduce_max(ious, -1)
# for the regression ,we need to know which boat we want to look at
argmax_iou_over_ground_truth = tf.argmax(ious, -1)

iou_average_for_summary.append(max_iou_over_ground_truth)

labels = tf.greater(max_iou_over_ground_truth, iou_positive_threshold)
labels = tf.cast(labels, tf.float32)

# We only deal in both losses with boxes with IOU above a upper threshold and
# below a lower threshold and so here we create a mask which will be 1 for
# all such iou scores and 0 for those inside the threshold so we can
# use it as a weighting for the losses
pos_labels = tf.cast(tf.greater(max_iou_over_ground_truth,
                                iou_positive_threshold), tf.float32)
neg_labels = tf.cast(tf.less(max_iou_over_ground_truth,
                             iou_negative_threshold), tf.float32)


pos_mask.append(pos_labels)
neg_mask.append(neg_labels)

#iou_mask_anchor = pos_labels + neg_labels
# iou_mask shape: [batch, 768, 768, n_proposal_boxes]
#iou_mask.append(tf.cast(iou_mask_anchor, tf.float32))

y_class.append(labels)
selected_boat_index.append(argmax_iou_over_ground_truth)

In [15]:
# Stack all the anchors together in the end this is then of shape [batch, 768, 768, n_anchor]
y_class = tf.stack(y_class, axis=-1)
selected_boat_index = tf.stack(selected_boat_index, axis=-1)

In [ ]:
# evaluate
np_y_class = tf_eval(y_class)

In [ ]:
print(np_y_class.shape) # (2, 768, 768, 1) where 1 is the number of anchors
print(np.sum(np_y_class[1,:])) # this should be zero and it is
print("sum:",np.sum(np_y_class[0,:,:,0]), "\tmax", np.amax(np_y_class[0,:,:,0]))

In [16]:
# Stack IOU masks
temp_pos_mask = tf.stack(pos_mask, -1)
temp_neg_mask = tf.stack(neg_mask, -1)
#iou_mask = tf.stack(iou_mask, -1)

In [ ]:
# evaluate
np_temp_pos_mask = tf_eval(temp_pos_mask)
np_temp_neg_mask = tf_eval(temp_neg_mask)

In [ ]:
for i in range(2):
    print("\n\nImage ",i)
    print("Fist check: are there any NaNs:",np.sum(np.isnan(np_temp_pos_mask[i])), np.sum(np.isnan(np_temp_neg_mask[i])))
    print('perc. positives:',np.sum(np_temp_pos_mask[i])/(768**2), "\tnum positives",np.sum(np_temp_pos_mask[i]))
    print('perc. negatives:',np.sum(np_temp_neg_mask[i])/(768**2), "\tnum negatives",np.sum(np_temp_neg_mask[i]))

In [ ]:
# IOUs take some time to evaluate so we are going to save the instance and run TF from here
path = "debug/np_temp_pos_mask.pkl"
np.save(path, np_temp_pos_mask)

In [ ]:
# checkpoint
y_class = tf.convert_to_tensor(np_y_class, tf.float32)
temp_pos_mask = tf.convert_to_tensor(np_temp_pos_mask, tf.float32)
temp_neg_mask = tf.convert_to_tensor(np_temp_neg_mask, tf.float32)

### filter_groundtruth_regression

In [17]:
def select_with_matrix_tf(tensor, indexer, batch_size):
    '''
    :param tensor: input tensor of shape [batch, h, w, depth, n_vals]
    :param indexer: indexing tensor of shape [batch, h, w] of ints which
    indicates which layer of the depth dimension to take
    :param batch_size: neccesary since first dim of tensor maybe none and cant set range(None) tensor
    :return: tensor of shape [batch, h, w, n_vals]
    '''
    batch_size_adaptive, h, w, depth, n_vals = tensor.get_shape().as_list()
    desired_shape = [batch_size, h, w]
    index_list = [
        tf.broadcast_to(tf.reshape(tf.range(batch_size, dtype=tf.int64), (-1, 1, 1)), desired_shape),
        tf.broadcast_to(tf.reshape(tf.range(h, dtype=tf.int64), (1, -1, 1)), desired_shape),
        tf.broadcast_to(tf.reshape(tf.range(w, dtype=tf.int64), (1, 1, -1)), desired_shape),
        indexer
    ]
    index = tf.stack(index_list, -1)
    return tf.gather_nd(tensor, index)

In [18]:
# Filter y_reg by which boxes are positive
y_reg_gt = []
k=0
# For each anchor run select_with_matrix_tf which filters the map of regression
# coordinates y_reg to the (per pixel) particular boat indicated by selected_boat_index
y_reg_gt_anchor = select_with_matrix_tf(tensor=y_reg,
                                        indexer=selected_boat_index[:,:,:, k],
                                        batch_size=2)
y_reg_gt.append(y_reg_gt_anchor)

y_reg_gt = tf.stack(y_reg_gt, -2)

In [ ]:
# evaluate
np_y_reg_gt = tf_eval(y_reg_gt,timed=1)

In [ ]:
print(np_y_reg_gt.shape, np.sum(np.isnan(np_y_reg_gt)))
# we only care for this function for the boat matrix
np.sum(np_y_reg_gt[0,:]==0)

In [ ]:
# checkpoint
y_reg_gt = tf.convert_to_tensor(np_y_reg_gt, tf.float32)

### sample

In [19]:
n_positive_samples = 100
n_negative_samples = 50
n_negative_samples_when_no_boats = 50

In [20]:
def np_sample(labels, label=1, n_samples=180):
    mask = np.zeros(labels.shape, dtype = float)
    # if we have np.sum(labels) = 0 we should have n_sample=0 as well
    #if n_samples > 0 & np.sum(labels) > 0:
    if n_samples > 0:
        idx1 = np.where(labels == label)
        idx1 = np.asarray(idx1)
        index = np.random.randint(idx1.shape[1], size=(n_samples))
        a = idx1[:, index]
        mask[a[0], a[1], a[2]] = label
        mask.astype(float)
    return mask

In [31]:
# counting the number of positive samples.
# if there are zero, then  we are in a "no_boats" batch image and sample consequently
# note: instead of tf.reduce_sum(self.y_map) > 0, we want tf.reduce_sum(self.y_map, [1,2,3])
# the latter gives us the sum of the ground truth per image
# if the sum is positive, then there are boats in the image and we want to sample some
# TODO: fix :)
class_mask = []
pos_mask = []
for i in range(2):
    sliced_temp_pos_mask = tf.slice(temp_pos_mask,begin=[i,0,0,0],size=[1,-1,-1,-1])
    summed_pos_mask = tf.reduce_sum(sliced_temp_pos_mask)
    n_positive_samples = tf.cond(summed_pos_mask > 0,
                             lambda: n_positive_samples,
                             lambda: 0)
    n_negative_samples = tf.cond(summed_pos_mask > 0,
                             lambda: n_negative_samples,
                             lambda: n_negative_samples_when_no_boats)

    # sampling
    # note that atm pos_sample applies the same sampling over the whole batch
    # refer to utils for the function
    # TODO: fix :)
    pos_sample = tf.py_func(np_sample, [temp_pos_mask[i], 1, n_positive_samples], tf.float64)
    pos_sample = tf.cast(pos_sample, tf.float32)
    #pos_mask = temp_pos_mask * pos_sample
    neg_sample = tf.py_func(np_sample, [temp_neg_mask[i], 1, n_negative_samples], tf.float64)
    neg_sample = tf.cast(neg_sample, tf.float32)
    #neg_mask = temp_neg_mask * neg_sample

    temp_class_mask = pos_sample + neg_sample
    class_mask.append(temp_class_mask)
    pos_mask.append(pos_sample)

In [32]:
# evaluate
np_summed_pos_mask = tf_eval(summed_pos_mask,timed=1)

time: 3.087090253829956


In [33]:
np_n_positive_samples = tf_eval(n_positive_samples,timed=1)

time: 1.5622100830078125


In [34]:
np_pos_sample = tf_eval(pos_sample,timed=1)

time: 1.1749320030212402


In [35]:
print(np_pos_sample.shape, np.sum(np.isnan(np_pos_sample)), np.sum(np_pos_sample))

(768, 768, 1) 0 0.0


In [36]:
class_mask = tf.stack(class_mask, 0)
pos_mask = tf.stack(pos_mask, 0)

In [37]:
# evaluate
np_class_mask = tf_eval(class_mask, timed=1)
np_pos_mask = tf_eval(pos_mask, timed=1)

time: 2.2557168006896973
time: 1.487652063369751


In [39]:
np.sum(np.isnan(np_class_mask)), np.sum(np.isnan(np_pos_mask))

(0, 0)

## Model

### Feature maps

In [46]:
# functions
def create_convolutional_layer(input,
                               num_input_channels,
                               conv_filter_size,
                               num_filters,
                               maxpool=1,
                               name=None):
    with tf.name_scope(name):
        ## We shall define the weights that will be trained using create_weights function.
        weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
        ## We create biases using the create_biases function. These are also trained.
        biases = create_biases(num_filters)

        ## Creating the convolutional layer
        layer = tf.nn.conv2d(input=input,
                             filter=weights,
                             strides=[1, 1, 1, 1],
                             padding='SAME')

        layer += biases

        ## We shall be using max-pooling.
        if maxpool == 1:
            layer = tf.nn.max_pool(value=layer,
                                   ksize=[1, 2, 2, 1],
                                   strides=[1, 2, 2, 1],
                                   padding='SAME')
        ## Output of pooling is fed to Relu which is the activation function for us.
        layer = tf.nn.relu(layer)

        return layer
    
def create_deconvolutional_layer(input, num_filters, name, upscale_factor):
    kernel_size = 2 * upscale_factor - upscale_factor % 2
    stride = upscale_factor
    strides = [1, stride, stride, 1]
    with tf.name_scope(name):
        # Shape of the input tensor
        batch_size = tf.shape(input)[0]
        input_size = input.get_shape().as_list()[1]
        h = input_size * stride
        w = input_size * stride
        # put everything together
        ds = [batch_size]
        ds.append(h)
        ds.append(w)
        ds.append(num_filters)
        deconv_shape = tf.stack(ds)

        filter_shape = [kernel_size, kernel_size, num_filters, num_filters]
        weights = get_bilinear_filter(filter_shape, upscale_factor)

        deconv = tf.nn.conv2d_transpose(value=input,
                                        filter=weights,
                                        output_shape=deconv_shape,
                                        strides=strides,
                                        padding='SAME')
    return deconv


def get_bilinear_filter(filter_shape, upscale_factor):
    # filter_shape is [width, height, num_in_channels, num_out_channels]
    kernel_size = filter_shape[1]
    ## Centre location of the filter for which value is calculated
    if kernel_size % 2 == 1:
        centre_location = upscale_factor - 1
    else:
        centre_location = upscale_factor - 0.5

    bilinear = np.zeros([filter_shape[0], filter_shape[1]])
    for x in range(filter_shape[0]):
        for y in range(filter_shape[1]):
            # Interpolation Calculation
            value = (1 - abs((x - centre_location) / upscale_factor)) * (
                        1 - abs((y - centre_location) / upscale_factor))
            bilinear[x, y] = value
    weights = np.zeros(filter_shape)
    for i in range(filter_shape[2]):
        weights[:, :, i, i] = bilinear
    init = tf.constant_initializer(value=weights,
                                   dtype=tf.float32)

    bilinear_weights = tf.get_variable(name="decon_bilinear_filter", initializer=init,
                                       shape=weights.shape)
    return bilinear_weights

def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))


def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

In [61]:
num_channels = 3
initial_im_size= 768
filter_size_conv1 =  3
num_filters_conv1 = 64
filter_size_conv2 = 3
num_filters_conv2 = 64
num_features = 64

In [54]:
layer_conv1 = create_convolutional_layer(input=tf.cast(x,tf.float32),
                                         num_input_channels=num_channels,
                                         conv_filter_size=filter_size_conv1,
                                         num_filters=num_filters_conv1,
                                         maxpool=0,
                                         name="conv_layer_1")


In [55]:

layer_conv2 = create_convolutional_layer(input=layer_conv1,
                                         num_input_channels=num_filters_conv1,
                                         conv_filter_size=filter_size_conv2,
                                         num_filters=num_filters_conv2,
                                         maxpool=0,
                                         name='conv_layer_2')

pool = tf.nn.max_pool(value=layer_conv2,
                      ksize=[1, 2, 2, 1],
                      strides=[1, 2, 2, 1],
                      padding='SAME')

feature_maps = create_deconvolutional_layer(input=pool,
                                            num_filters=num_filters_conv2,
                                            name='deconvolution',
                                            upscale_factor=2)

In [58]:
# eval 
np_feature_maps = tf_eval(feature_maps,timed=1)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
time: 18.075649976730347


In [60]:
np_feature_maps.shape, np.sum(np.isnan(np_feature_maps))

((2, 768, 768, 64), 0)

### region_proposal_network
#### CNN

In [68]:
def create_convolution(input,
                       num_input_channels,
                       conv_filter_size,
                       num_filters,
                       stride=1,
                       data_format="NHWC"):
    '''
    Simplified version of create_convolutional_layer that doesn't include max pooling or activation function
    :param input: input tensor
    :param num_input_channels: number of input channels
    :param conv_filter_size: width and height of square conv filter
    :param num_filters: number of filters to be applied
    :param stride: scalar stride size for both width and height
    :param data_format: see tensorflow documentation
    :return:
    '''
    if data_format == "NHWC":
        strides = [1, stride, stride, 1]
    elif data_format == "NCHW":
        strides = [1, 1, stride, stride]
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)


    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                         filter=weights,
                         strides=strides,
                         padding='SAME')

    layer += biases

    return layer

In [66]:
window_size = 3
sliding_hidden_layer_size = 64
n_proposal_boxes = 1

In [70]:

#with tf.name_scope('sliding_window'):
window_outputs = create_convolution(input=feature_maps,
                                    num_input_channels=num_features,
                                    conv_filter_size=window_size,
                                    num_filters=sliding_hidden_layer_size,
                                    stride=1,
                                    data_format="NHWC")

#with tf.name_scope('classification_layer'):
class_scores = create_convolution(input=window_outputs,
                                       num_input_channels=sliding_hidden_layer_size,
                                       conv_filter_size=1,
                                       num_filters=n_proposal_boxes,
                                       stride=1,
                                       data_format="NHWC")
#with tf.name_scope('regression_layer'):
# reg_outputs: [batch_size, 768, 768, 4*self.config.n_proposal_boxes]
reg_outputs = create_convolution(input=window_outputs,
                                     num_input_channels=sliding_hidden_layer_size,
                                     conv_filter_size=1,
                                     num_filters=4*n_proposal_boxes,
                                     stride=1,
                                     data_format="NHWC")

# self.reg_scores: [batch_size, 768, 768, self.config.n_proposal_boxes, 4]
reg_scores = tf.reshape(tensor=reg_outputs,
                             shape=[-1, 768, 768, n_proposal_boxes, 4],
                             name='reshape_regression_outputs')

In [65]:
y_reg_gt.shape, reg_anchors.shape

(TensorShape([Dimension(2), Dimension(768), Dimension(768), Dimension(1), Dimension(4)]),
 TensorShape([Dimension(2), Dimension(768), Dimension(768), Dimension(1), Dimension(4)]))

In [71]:
# eval 
np_class_scores = tf_eval(class_scores, timed=1)
np_reg_scores = tf_eval(reg_scores,timed=1)

time: 26.002481698989868
time: 26.203938961029053


In [74]:
print(np_class_scores.shape, np_reg_scores.shape)
print(np.sum(np.isnan(np_class_scores)), np.sum(np.isnan(np_reg_scores)))

(2, 768, 768, 1) (2, 768, 768, 1, 4)
0 0


In [75]:
class_scores = tf.convert_to_tensor(np_class_scores, tf.float32)
reg_scores = tf.convert_to_tensor(np_reg_scores, tf.float32)

## Loss

In [76]:
sigmoid_ce = tf.nn.sigmoid_cross_entropy_with_logits(labels=y_class,
                                                     logits=class_scores)
# Remember that we only look at positive (> upper iou thresh) and negative (< iou thresh) boxes
print('IOU Mask Shape', class_mask.shape)
masked_signoid_ce = tf.multiply(class_mask, sigmoid_ce)

classification_loss_per_sample = tf.reduce_sum(masked_signoid_ce, axis = [1,2,3])
classification_loss = tf.reduce_mean(classification_loss_per_sample)

IOU Mask Shape <unknown>


In [77]:
# evaluate
np_classification_loss = tf_eval(classification_loss, timed =1)

time: 6.103011846542358


In [78]:
np_classification_loss

5.5542912

### create_adjusted_coordinates

In [80]:
# t_x = (x_predict - x_anchor)/ w_anchor
epsilon = 0.001
t_x = (reg_scores[:, :, :, :, 0] - reg_anchors[:, :, :, :, 0]) / (reg_anchors[:, :, :, :, 2] + epsilon)
t_x_star = (y_reg_gt[:, :, :, :, 0] - reg_anchors[:, :, :, :, 0]) / (reg_anchors[:, :, :, :, 2] + epsilon)

In [84]:
# t_y = (y_predict - y_anchor)/ h_anchor
t_y = (reg_scores[:, :, :, :, 1] - reg_anchors[:, :, :, :, 1]) / (reg_anchors[:, :, :, :, 3] + epsilon)
t_y_star = (y_reg_gt[:, :, :, :, 1] - reg_anchors[:, :, :, :, 1]) / (reg_anchors[:, :, :, :, 3] + epsilon)
# t_w = log(w_predict / w_anchor)
t_w_star = tf.log(y_reg_gt[:, :, :, :, 2] / (reg_anchors[:, :, :, :, 2]+ epsilon))
# t_w = tf.maximum(tf.log(self.reg_scores[:, :, :, :, 2] / reg_anchors[:, :, :, :, 2]),
#                  tf.cast(tf.fill(dims=tf.shape(t_w_star), value=-100000), dtype=tf.float32))
t_w = tf.log(reg_scores[:, :, :, :, 2] / (reg_anchors[:, :, :, :, 2] + epsilon))
# tf.constant(-100000, shape=tf.shape(t_w_star)))
# t_h = log(h_predict / h_anchor)
t_h_star = tf.log(y_reg_gt[:, :, :, :, 3] / (reg_anchors[:, :, :, :, 3] + epsilon))
# t_h = tf.maximum(tf.log(self.reg_scores[:, :, :, :, 3] / reg_anchors[:, :, :, :, 3]),
#                  tf.cast(tf.fill(dims=tf.shape(t_w_star), value=-100000), dtype=tf.float32))
t_h = tf.log(reg_scores[:, :, :, :, 3] / (reg_anchors[:, :, :, :, 3] + epsilon))

In [81]:
np_t_x = tf_eval(t_x,timed=1)

time: 4.759867906570435


In [83]:
np.sum(np.isnan(np_t_x))

0

In [85]:
np_t_w = tf_eval(t_w, timed =1)

time: 4.208181858062744


In [87]:
np.sum(np.isnan(np_t_w))

1105928

In [88]:
t_w = tf.maximum(tf.log(reg_scores[:, :, :, :, 2] / reg_anchors[:, :, :, :, 2]),
                 tf.cast(tf.fill(dims=tf.shape(t_w_star), value=-100000), dtype=tf.float32))

In [89]:
np_t_w = tf_eval(t_w, timed =1)

time: 6.304682970046997


In [90]:
np.sum(np.isnan(np_t_w))

1105928

In [91]:
t_w = tf.log(reg_scores[:, :, :, :, 2]+ epsilon / (reg_anchors[:, :, :, :, 2] + epsilon))
np_t_w = tf_eval(t_w, timed =1)
print(np.sum(np.isnan(np_t_w)))

time: 4.869354009628296
1105920


In [92]:
temp = reg_scores[:, :, :, :, 2] / reg_anchors[:, :, :, :, 2]
np_temp = tf_eval(temp)

In [95]:
print(np.sum(np.isnan(np_temp)))
np_temp.shape,reg_scores[:, :, :, :, 2].shape

0


((2, 768, 768, 1),
 TensorShape([Dimension(2), Dimension(768), Dimension(768), Dimension(1)]))

In [98]:
t_w = tf.log(tf.maximum(reg_scores[:, :, :, :, 2],
                        tf.cast(tf.fill(dims=tf.shape(reg_scores[:, :, :, :, 2]), value=epsilon), dtype=tf.float32))
                        / (reg_anchors[:, :, :, :, 2] + epsilon))

In [99]:
np_t_w = tf_eval(t_w, timed =1)
print(np.sum(np.isnan(np_t_w)))

time: 5.8116090297698975
0


In [100]:
# t_y = (y_predict - y_anchor)/ h_anchor
t_y = (reg_scores[:, :, :, :, 1] - reg_anchors[:, :, :, :, 1]) / (reg_anchors[:, :, :, :, 3] + epsilon)
t_y_star = (y_reg_gt[:, :, :, :, 1] - reg_anchors[:, :, :, :, 1]) / (reg_anchors[:, :, :, :, 3] + epsilon)
# t_w = log(w_predict / w_anchor)
t_w_star = tf.log(y_reg_gt[:, :, :, :, 2] / (reg_anchors[:, :, :, :, 2]+ epsilon))
# t_w = tf.maximum(tf.log(self.reg_scores[:, :, :, :, 2] / reg_anchors[:, :, :, :, 2]),
#                  tf.cast(tf.fill(dims=tf.shape(t_w_star), value=-100000), dtype=tf.float32))
reg_adj_1 = tf.maximum(reg_scores[:, :, :, :, 2],
           tf.cast(tf.fill(dims=tf.shape(reg_scores[:, :, :, :, 2]), value=epsilon), dtype=tf.float32))

t_w = tf.log(reg_adj_1 / (reg_anchors[:, :, :, :, 2] + epsilon))
# tf.constant(-100000, shape=tf.shape(t_w_star)))
# t_h = log(h_predict / h_anchor)
t_h_star = tf.log(y_reg_gt[:, py:, :, :, 3] / (reg_anchors[:, :, :, :, 3] + epsilon))
# t_h = tf.maximum(tf.log(self.reg_scores[:, :, :, :, 3] / reg_anchors[:, :, :, :, 3]),
#                  tf.cast(tf.fill(dims=tf.shape(t_w_star), value=-100000), dtype=tf.float32))
reg_adj_2 = tf.maximum(reg_scores[:, :, :, :, 3],
           tf.cast(tf.fill(dims=tf.shape(reg_scores[:, :, :, :, 3]), value=epsilon), dtype=tf.float32))

t_h = tf.log(reg_adj_2 / (reg_anchors[:, :, :, :, 3] + epsilon))

In [101]:
y_reg_loss_pred = tf.stack([t_x, t_y, t_w, t_h], axis=4)
y_reg_loss_gt = tf.stack([t_x_star, t_y_star, t_w_star, t_h_star], axis=4)


regression_loss_per_pixel = tf.losses.mean_squared_error(labels=y_reg_loss_gt,
                                                     predictions=y_reg_loss_pred,
                                                     reduction=tf.losses.Reduction.NONE)


In [103]:
np_regression_loss_per_pixel = tf_eval(regression_loss_per_pixel, timed=1)

time: 19.68030881881714


In [104]:
np.sum(np.isnan(np_regression_loss_per_pixel))

0